In [33]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-10-23 12:27:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-10-23 12:27:54 (32.1 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [34]:
import minsearch

In [35]:
import json

In [36]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [37]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [38]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [39]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [40]:
q = 'the course has already started, can I still enroll?'

In [41]:

index.fit(documents)

In [42]:
import google.generativeai as genai
from dotenv import load_dotenv

import os

load_dotenv() 
gemma_client = genai.configure(api_key=os.environ["GEMMA_API_KEY"])


In [54]:
# Assuming you have initialized Gemma client
model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content(q)
# Assuming 'response' is your GenerateContentResponse object
response.text

"Please provide me with more information so I can help you! I need to know:\n\n* **What course are you interested in?** (e.g., online coding bootcamp, university course, etc.)\n* **Where is the course offered?** (e.g., specific platform, university name, etc.)\n\nOnce I have this information, I can look into the course's enrollment policies and let you know if it's still possible to join. \n"

In [55]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [56]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [57]:
def llm(prompt):

    response = model.generate_content(prompt)

    return response.text

In [73]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [74]:
rag(query)

'The FAQ database does not provide information on how to run Kafka in general. It only provides information on how to run Kafka producers, consumers, kstreams, and other components in specific programming languages. \n'

In [75]:
rag('the course has already started, can I still enroll?')

"The FAQ mentions that even if you don't register before the start date, you can still submit homeworks.  However, it also says that there are deadlines for turning in the final projects. \n"

In [76]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [83]:
from elasticsearch import Elasticsearch

# Create a connection to the Elasticsearch cluster
es = Elasticsearch(
    ["http://localhost:9200"],  # Replace with your Elasticsearch host
    http_auth=("username", "password"),  # Optional, for secured clusters
    timeout=60,  # Timeout setting (optional)
)

# Check if the connection is successful by pinging the cluster
if es.ping():
    print("Successfully connected to Elasticsearch")
else:
    print("Could not connect to Elasticsearch")

# Example: Index a document
doc = {
    "author": "Amber",
    "text": "Learning Elasticsearch with Python.",
    "timestamp": "2024-10-23T12:00:00"
}

response = es.index(index="my_index", id=1, document=doc)
print(response)


/var/folders/0m/hd5nyhdj19b3dcxl72fwby3c0000gn/T/ipykernel_80783/328773094.py:4: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(
/var/folders/0m/hd5nyhdj19b3dcxl72fwby3c0000gn/T/ipykernel_80783/328773094.py:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Successfully connected to Elasticsearch
{'_index': 'my_index', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [84]:
es_client = Elasticsearch('http://localhost:9200') 

In [85]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [86]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [87]:
from tqdm.auto import tqdm

In [88]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

python(84327) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  0%|          | 0/948 [00:00<?, ?it/s]

In [89]:
query = 'I just disovered the course. Can I still join it?'

In [90]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [91]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [92]:
rag(query)

"Yes, you can still join the course even if you don't register. You are still eligible to submit homeworks. However, be aware that there will be deadlines for turning in the final projects. \n"